### ***Retrieval-Augmented Generation.***
It's a way to improve AI-generated answers by letting the model search for real information before answering.

This project is an AI-powered Tutor designed to help school students learn by answering their questions using real-world information from Wikipedia.

It uses a powerful combination of:

#### ***Retrieval →*** Finds facts from Wikipedia

#### ***Augmentation →*** Merges them with the student's question

#### ***Generation*** → Produces a clear, personalized answer using an AI model

### ***Helpful for Students:***
Explains topics clearly using trusted sources

Reduces confusion with accurate, direct answers

Can be used for homework help, exam prep, or curiosity-based learning

Always updated because it uses live Wikipedia content

### ***Installation***

In [1]:
!pip install transformers gradio wikipedia --quiet

###***Transformers***
Fetch models from Hugging Face.

we could easily use pre-trained language models like:

Mistral, FLAN-T5, GPT-2, BERT, etc.

***Key functions:***

***AutoTokenizer*** – converts text into model-readable tokens.

***AutoModelForCausalLM*** – loads the language model itself.

***gradio***

Builds the user interface (text input/output fields)

***wikipedia***

Fetches real-world knowledge summaries from Wikipedia

In [2]:
!pip install  accelerate bitsandbytes --quiet


###***bitsandbytes***
This library enables 4-bit and 8-bit quantization for large models.

Reduces memory usage (VRAM) drastically

Load big models (like Mistral-7B) on limited GPUs

Use load_in_4bit=True when calling from_pretrained(...)

###***accelerate***
Library by Hugging Face to manage device placement (CPU/GPU/TPU) and distributed training easily.

It handles complex stuff like:

Moving models to the correct device

Mixed precision (like float16)

Training on multiple GPUs or even TPUs

### ***Importing Modules***

In [3]:
import wikipedia
import re
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import gradio as gr

Model Loaders

## ***Hugging Face***

Hugging Face Hub is an online platform where AI models like Mistral are stored.

Some models require authentication(like private or gated models(Mistral))

Get a token from your Hugging Face account → used to unlock the models

In [4]:
login()  # It will securely ask for the token without hardcoding

# logs into the Hugging Face Hub using personal access token.


## ***Loading a full-sized instruction-tuned LLM (Mistral 7B) from Hugging Face***

In [5]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", #Auto-loads on GPU/CPU
    load_in_4bit=True, # Loads a quantized version for low RAM usage
    torch_dtype=torch.float16, #Uses half-precision to save VRAM
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##***The tokenizer's job***
 convert your input string → tokens (numbers the model understands).*italicized text*

## ***Cleaning the Context***
cleans raw Wikipedia text or PDF content before feeding it to the LLM — so the model doesn't get confused by LaTeX, special symbols, or messy spacing.

In [6]:
def clean_context(text):
    # Remove LaTeX-like symbols(\frac, \begin, \alpha)
    text = re.sub(r'\\[a-zA-Z]+', '', text)

    # Remove unwanted characters but keep common punctuation
    text = re.sub(r'[^a-zA-Z0-9.,:;()\-\+\s]', '', text)

    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text).strip()

    return text


## ***Fetching Context from Wikipedia***

In [7]:
def get_context(query):
    try:
        summary = wikipedia.summary(query, sentences=25) #summary of 25 sentences
        return clean_context(summary)
    except wikipedia.exceptions.DisambiguationError as e: #fault-tolerant
        return f"Ambiguous topic. Try one of: {e.options[:3]}"
    except wikipedia.exceptions.PageError: #no page that matches the query
        return "Page not found. Try a more specific topic."
    except Exception as e:
        return f"Error: {str(e)}"

#### ***Disambiguation***

*   Happens if the query is vague (e.g., "Apple")

*   Returns top 3 suggestions (e.g., "Apple Inc.", "Apple (fruit)", "Apple Records").



### ***Generate Simple Explanations and Quizzes Using a Language Model***

In [8]:
def ask_mistral(query, context):
    prompt = f"""You are a helpful AI tutor designed to explain concepts in a very simple and clear way for school students.

Context:
{context}

Your task:
1. Explain the topic "{query}" to a school student.
2. Use exactly 10 bullet points.
3. Each bullet point should be short, easy to understand, and focused on one important fact.
4. Avoid complex terms, technical phrases, or long paragraphs.

Then, generate 3 quiz questions based on this explanation.

Respond only with the explanation and the quiz in this format:
1. ...

"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
# Converts model’s tokens back to readable text.
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


    start_idx = response.find("Explanation:")
    if start_idx != -1:
        response = response[start_idx:]


    explanation_part, quiz_part = response.split("Quiz Questions:", 1)
    explanation = explanation_part.replace("Explanation:", "").strip()
    quiz = quiz_part.strip()



    return explanation, quiz


***Purpose:***
It creates a simple, school-level explanation and quiz for any topic using a language model.

***Inputs:***
Takes two inputs — query (the topic) and context (background info to guide the explanation).

***Prompt Building:***
Constructs a clear and structured prompt with rules: 10 easy bullet points and 3 quiz questions.

***Model Response:***
Sends the prompt to a language model, which returns a generated explanation and quiz.

***Output:***
Extracts and returns the explanation and quiz separately as two string values.

In [11]:
def tutor_pipeline(topic):
    context = get_context(topic)
    explanation, quiz = ask_mistral(topic, context)
    return explanation, quiz

### ***AI Tutor UI with Gradio***

In [12]:
import gradio as gr

with gr.Blocks(
    theme=gr.themes.Soft(primary_hue="indigo", secondary_hue="blue"),
    title="AI Tutor 📘"
) as iface:

    # Header
    gr.Markdown("""
    <div style='text-align: center; padding: 1.5rem; background-color: #e0edff; border-radius: 12px;'>
        <h1 style='font-size: 2.5rem; color: #1e3a8a;'>📘 AI Tutor</h1>
        <p style='font-size: 1.2rem; color: #334155; max-width: 600px; margin: auto;'>
            Type a topic and get a 10-point explanation + quiz — Perfect for school students!
        </p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            topic_input = gr.Textbox(
                label="🔍 Enter a Topic",
                placeholder="E.g. Reproduction, Volcanoes, Photosynthesis...",
                lines=1,
            )
            submit_button = gr.Button("💡 Explain It")

        with gr.Column(scale=2):
            output_explanation = gr.Textbox(
                label="📘 Easiest Explanation",
                lines=12,
                show_copy_button=True,
                interactive=True
            )

            output_quiz = gr.Textbox(
                label="❓ Quiz Questions",
                lines=5,
                show_copy_button=True,
                interactive=True
            )

    # Link the button to your pipeline
    submit_button.click(
        fn=tutor_pipeline,
        inputs=topic_input,
        outputs=[output_explanation, output_quiz]
    )

    # Footer
    gr.Markdown("""
    <div style='text-align: center; margin-top: 2rem; font-size: 0.9rem; color: #64748b;'>
        Built for curious minds — Powered by Mistral + Wikipedia
    </div>
    """)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a2caf3f38fa34329b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
